In [2]:
#Importamos librerias a utilizar
import pandas as pd
import requests


In [3]:
#Buscamos la palabra astro
palabra = 'astro'
#variable con el url
direccion = 'https://es.wikipedia.org/w/rest.php/v1/search/title'
#creamos nuestro duccionario
diccionario = {'q': palabra, 'limit': 20}
#Realizamos la petición
request = requests.get(direccion, params=diccionario)
#Verificamos el status
request.status_code

200

In [4]:
#Verificamos en un dataframe
df = pd.json_normalize(request.json()['pages'])
df

,id,key,title,excerpt,matched_title,description,thumbnail,thumbnail.mimetype,thumbnail.size,thumbnail.width,thumbnail.height,thumbnail.duration,thumbnail.url
0,59546,Objeto_astronómico,Objeto astronómico,Astro,Astro,"cuerpo físico de tamaño, masa o función astron...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,261,Astronomía,Astronomía,Astronomía,None,ciencia que estudia los cuerpos celestes del u...,NaN,image/jpeg,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
2,266,Astrología,Astrología,Astrología,None,forma de adivinación imaginaria basada en el e...,NaN,image/jpeg,NaN,60.0,78.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
3,257842,Astrología_china,Astrología china,Astrología china,None,horóscopo chino,NaN,image/png,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
4,9149789,Astronomía_maya,Astronomía maya,Astronomía maya,None,dioses de esa epoca,NaN,image/jpeg,NaN,60.0,87.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
5,15216,Astronáutica,Astronáutica,Astronáutica,None,teoría y práctica de la navegación fuera de la...,NaN,image/jpeg,NaN,60.0,71.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
6,6007265,Astrobiología,Astrobiología,Astrobiología,None,disciplina que estudia la existencia de vida f...,NaN,image/gif,NaN,60.0,34.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
7,3323,Astronauta,Astronauta,Astronauta,None,miembro de la tripulación o personal de una na...,NaN,image/jpeg,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
8,7411986,Astro_(grupo_musical),Astro (grupo musical),Astro (grupo musical),None,grupo musical masculino surcoreano,NaN,image/png,NaN,60.0,28.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
9,1779766,Astronomía_en_Chile,Astronomía en Chile,Astronomía en Chile,None,None,NaN,image/jpeg,NaN,60.0,54.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...


In [5]:
# Obtener los títulos y las páginas en listas
Lista_Titulos = df['title'].tolist()
Lista_Paginas = df['id'].tolist()

# Crear la lista de extractos
extracts_list = []

# Realizar las solicitudes a la API de Wikipedia
for i in range(len(Lista_Paginas)):
    page = Lista_Paginas[i]
    title = Lista_Paginas[i]
    url = f'https://es.wikipedia.org/w/api.php?action=query&format=json&pageids={page}&prop=extracts&explaintext=True&exintro=True'
    request_wiki = requests.get(url)
    data = request_wiki.json()
    extract = data['query']['pages'][str(page)]['extract']
    extracts_list.append(extract)

# Crear el diccionario y convertirlo en DataFrame
extracts_dict = {'title': Lista_Titulos, 'page_id': Lista_Paginas, 'extract': extracts_list}
extracts_df = pd.DataFrame(extracts_dict)

In [7]:
# Verificar si las palabras "ciencia" o "física" están en la columna "extract"
BCF = (extracts_df['extract'].str.contains('ciencia', case=False) | extracts_df['extract'].str.contains('física', case=False)).astype(int).tolist()
BCF

[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]

In [9]:
# Convertir la lista de resultados a una serie de pandas
Fisica_Ciencia_Binary = pd.Series(BCF)

# Unir la serie de resultados al dataframe original y renombrar a 'contiene_palabra'
Resultados = pd.concat([extracts_df, Fisica_Ciencia_Binary.rename('contiene_palabra')], axis=1)
Resultados

,title,page_id,extract,contiene_palabra,contiene_palabra
0,Objeto astronómico,59546,Un objeto astronómico es una entidad física si...,1,1
1,Astronomía,261,La astronomía (del griego άστρον [ástron] 'est...,1,1
2,Astrología,266,"La astrología, en su acepción más amplia, es u...",1,1
3,Astrología china,257842,La astrología china es la astrología del calen...,0,0
4,Astronomía maya,9149789,La astronomía maya forma parte de una tradició...,0,0
5,Astronáutica,15216,La astronáutica es la teoría y práctica de la ...,1,1
6,Astrobiología,6007265,La astrobiología es la rama de la biología y l...,1,1
7,Astronauta,3323,Un astronauta[1]​ o cosmonauta[2]​ es una pers...,0,0
8,Astro (grupo musical),7411986,"Astro (en hangul, 아스트로) (estilizado como ASTRO...",0,0
9,Astronomía en Chile,1779766,La astronomía en Chile ha sido desarrollada en...,1,1
